In [41]:
import json
import pandas as pd
import re
import random
from scipy import sparse
import numpy as np
from pymongo import MongoClient
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import sys
sys.path.append('../machine_learning')
import yelp_ml as yml
reload(yml)
from gensim import corpora, models, similarities, matutils
import tqdm

In [2]:
lh_neg = open('../input/negative-words.txt', 'r').read()
lh_neg = lh_neg.split('\n')
lh_pos = open('../input/positive-words.txt', 'r').read()
lh_pos = lh_pos.split('\n')
users = json.load(open("cleaned_large_user_dictionary.json"))
word_list = list(set(lh_pos + lh_neg))

In [3]:
ip = '184.73.129.244'
conn = MongoClient(ip, 27017)
db = conn.get_database('cleaned_data')
reviews = db.get_collection('restaurant_reviews')
users_results = pd.DataFrame()
user_id = []

In [43]:
for j in tqdm.tqdm(range(0, len(users.keys()[0:1]))):
	
	user_id.append(users[users.keys()[i]])

	user_df = yml.make_user_df(users[users.keys()[i]])
	business_ids = list(set(user_df['biz_id']))
	restreview = {}

	for i in range(0, len(business_ids)):
		rlist = []
		for obj in reviews.find({'business_id':business_ids[i]}):
			rlist.append(obj)
		restreview[business_ids[i]] = rlist
	restaurant_df = yml.make_biz_df(users.keys()[j], restreview)
	
	#Create a training and test sample from the user reviewed restaurants
	split_samp = .20
	random_int = random.randint(1, len(business_ids)-1)
	len_random = int(len(business_ids) * split_samp)
	test_set = business_ids[random_int:random_int+len_random]
	training_set = business_ids[0:random_int]+business_ids[random_int+len_random:len(business_ids)]
	train_reviews, train_ratings = [], []
	
	#Create a list of training reviews and training ratings
	for rest_id in training_set:
		train_reviews.extend(list(user_df[user_df['biz_id'] == rest_id]['review_text']))
		train_ratings.extend(list(user_df[user_df['biz_id'] == rest_id]['rating']))
	
	#Transform the star labels into a binary class problem, 0 if rating is < 4 else 1
	train_labels = [1 if x >=4 else 0 for x in train_ratings]
	
	#Fit LSI model and return number of LSI topics
	lsi, topics, dictionary = yml.fit_lsi(train_reviews)

	#Create a training and test sample from the user reviewed restaurants
	split_samp = .30
	random_int = random.randint(1, len(business_ids)-1)
	len_random = int(len(business_ids) * split_samp)
	test_set = business_ids[random_int:random_int+len_random]
	training_set = business_ids[0:random_int]+business_ids[random_int+len_random:len(business_ids)]
	sub_train_reviews, train_labels, train_reviews, train_ratings = [], [], [], []

	#Create a list of training reviews and training ratings
	for rest_id in training_set:
		train_reviews.append((user_df[user_df['biz_id'] == rest_id]['review_text'].iloc[0],
								 user_df[user_df['biz_id'] == rest_id]['rating'].iloc[0]))

	#Create an even sample s.t. len(positive_reviews) = len(negative_reviews)
	sample_size = min(len([x[1] for x in train_reviews if x[1] < 4]),
						  len([x[1] for x in train_reviews if x[1] >= 4]))
	
	bad_reviews = [x for x in train_reviews if x[1] < 4]
	good_reviews = [x for x in train_reviews if x[1] >= 4]

	for L in range(0, int(float(sample_size)/float(2))):
		sub_train_reviews.append(bad_reviews[L][0])
		sub_train_reviews.append(good_reviews[L][0])
		train_labels.append(bad_reviews[L][1])
		train_labels.append(good_reviews[L][1])
        
	#Make the train labels binary
	train_labels = [1 if x >=4 else 0 for x in train_labels]
    
	#Make a FeatureUnion object with the desired features then fit to train reviews
	test_results = {}
	feature_selection = [(True, True, False), (True,False,False), (False,True,True), (True, True, True)]

	for feature in feature_selection:
		#Make a FeatureUnion object with the desired features then fit to train reviews
		comb_features = yml.make_featureunion(sent_percent=feature[0], tf = feature[1], lda = feature[2])
		comb_features.fit(sub_train_reviews)
		train_features = comb_features.transform(sub_train_reviews)
		train_lsi = yml.get_lsi_features(sub_train_reviews, lsi, topics, dictionary)
		train_features = sparse.hstack((train_features, train_lsi))
		train_features = train_features.todense()

		#Fit LSI model and return number of LSI topics
		lsi, topics, dictionary = yml.fit_lsi(sub_train_reviews)
		
		#fit each model in turn 
		model_runs = [(True, False, False), (False, True, False), 
					  (False, False, True)]

		for model_run in model_runs:
			clf = yml.fit_model(train_features, train_labels, svm_clf = model_run[0], 
							RandomForest = model_run[1], nb = model_run[2])
			threshold = 0.7
			error = yml.test_user_set(test_set, clf, restaurant_df, user_df, comb_features, threshold, lsi, topics, dictionary)
			test_results[(feature, clf)] = (yml.get_log_loss(error), yml.get_accuracy_score(error), yml.get_precision_score(error))

string_keys_dict = {}
for key in test_results.keys():
    string_keys_dict[str(key)] = test_results[key]
    
with open('test_results.json', 'wb') as fp:
	json.dump(string_keys_dict, fp)







  0%|          | 0/1 [00:00<?, ?it/s]

(True, True, False)
(True, False, False)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


(False, True, False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=100, max_features='auto', max_leaf_nodes=50,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
(False, False, True)
GaussianNB(priors=None)
(True, False, False)


/Users/robertsonwang/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


(True, False, False)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
(False, True, False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=100, max_features='auto', max_leaf_nodes=50,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
(False, False, True)
GaussianNB(priors=None)
(False, True, True)


...

(True, False, False)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
(False, True, False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=100, max_features='auto', max_leaf_nodes=50,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
(False, False, True)
GaussianNB(priors=None)
(True, True, True)


/Users/robertsonwang/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


(True, False, False)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
(False, True, False)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=100, max_features='auto', max_leaf_nodes=50,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
(False, False, True)
GaussianNB(priors=None)








100%|██████████| 1/1 [13:13<00:00, 793.01s/it]







TypeError: key ((True, True, False), GaussianNB(priors=None)) is not a string

In [61]:
new_dict = {}
for key in test_results.keys():
    new_dict[str(key)] = test_results[key]
    
with open('test.json', 'wb') as fp:
	json.dump(new_dict, fp)

In [63]:
test = json.load(open('test.json', 'r'))